In [ ]:
# notebooks/03-feature-engineering.ipynb

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# --- Kết nối tới Database (tương tự notebook trước) ---
load_dotenv()
db_user = os.getenv('MYSQL_USER')
db_password = os.getenv('MYSQL_PASSWORD')
db_host = os.getenv('MYSQL_HOST')
db_port = os.getenv('MYSQL_PORT')
db_name = os.getenv('MYSQL_DATABASE')
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# --- Viết Master Query ---
# Câu lệnh này sẽ JOIN hầu hết các bảng quan trọng lại với nhau
master_query = """
SELECT
    o.order_id,
    -- Target variable
    r.review_score,
    
    -- Features từ order_items
    oi.price,
    oi.freight_value,
    
    -- Features từ orders
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    
    -- Features từ products
    p.product_category_name,
    p.product_weight_g,
    p.product_length_cm,
    p.product_height_cm,
    p.product_width_cm,
    
    -- Features từ sellers
    s.seller_state,
    
    -- Features từ customers
    c.customer_state
FROM
    orders o
JOIN order_reviews r ON o.order_id = r.order_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN sellers s ON oi.seller_id = s.seller_id
JOIN customers c ON o.customer_id = c.customer_id
WHERE
    o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL;
"""

# Thực thi query và load vào DataFrame
print("Đang tải dữ liệu từ Master Query...")
df = pd.read_sql(master_query, engine)
print(f"Tải thành công {len(df)} dòng dữ liệu.")
display(df.head())

In [ ]:
# Chuyển đổi các cột ngày tháng sang kiểu datetime
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

# --- Tạo các feature về thời gian vận chuyển ---

# 1. Thời gian giao hàng thực tế (ngày)
df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 2. Thời gian giao hàng dự kiến (ngày)
df['estimated_delivery_days'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 3. Độ trễ so với dự kiến (ngày) -> Feature cực kỳ quan trọng!
# Số dương nghĩa là giao trễ, số âm là giao sớm
df['delivery_delay'] = df['delivery_days'] - df['estimated_delivery_days']

# 4. Feature nhị phân: có giao trễ hay không?
df['is_late'] = (df['delivery_delay'] > 0).astype(int)

# --- Tạo các feature khác ---

# 5. Tỷ lệ phí vận chuyển trên giá sản phẩm
# Thêm 1e-6 để tránh chia cho 0
df['freight_ratio'] = df['freight_value'] / (df['price'] + 1e-6)

# 6. Kích thước thể tích sản phẩm
df['product_volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# 7. Ngày mua hàng trong tuần (Thứ 2=0, Chủ Nhật=6)
df['purchase_day_of_week'] = df['order_purchase_timestamp'].dt.dayofweek

print("Đã tạo xong các feature mới. Xem thử một vài dòng:")
display(df[['review_score', 'delivery_days', 'estimated_delivery_days', 'delivery_delay', 'is_late', 'freight_ratio']].head())

In [ ]:
# Lựa chọn các cột cuối cùng cho mô hình
# Chúng ta sẽ loại bỏ các cột ID, các cột ngày tháng gốc và các cột đã dùng để tạo feature
features_to_keep = [
    # Target
    'review_score',
    
    # Numeric features
    'price',
    'freight_value',
    'product_weight_g',
    'product_volume_cm3',
    'delivery_days',
    'delivery_delay',
    'freight_ratio',
    'purchase_day_of_week',
    
    # Categorical features
    'product_category_name',
    'seller_state',
    'customer_state'
]

df_model_ready = df[features_to_keep].copy()

# Xử lý missing values (nếu có)
# Ví dụ: điền giá trị trung bình cho các cột số
for col in df_model_ready.select_dtypes(include=np.number).columns:
    if df_model_ready[col].isnull().any():
        median_val = df_model_ready[col].median()
        df_model_ready[col].fillna(median_val, inplace=True)

# Ví dụ: điền 'unknown' cho các cột category
for col in df_model_ready.select_dtypes(include='object').columns:
    if df_model_ready[col].isnull().any():
        df_model_ready[col].fillna('unknown', inplace=True)
        
print("DataFrame đã sẵn sàng cho mô hình:")
df_model_ready.info()

In [ ]:
# Tạo thư mục data/processed nếu chưa có
processed_data_path = '../data/processed'
os.makedirs(processed_data_path, exist_ok=True)

# Lưu file
output_path = os.path.join(processed_data_path, 'model_ready_data.csv')
df_model_ready.to_csv(output_path, index=False)

print(f"Đã lưu bộ dữ liệu đã xử lý tại: {output_path}")